# 🚀 PolyScale-FL Demonstration Notebook
### *Decentralized Federated Learning with Blockchain + IPFS + Secure Aggregation*

This notebook demonstrates how the full project works:

✔ Load dataset (CIFAR-10 / MNIST)
✔ Initialize FL clients
✔ Local training
✔ Secure aggregation
✔ Upload model updates to IPFS
✔ Commit update hash to blockchain smart contract
✔ Aggregator produces a global model

---

In [1]:
# ------------------------------------------
# Imports
# ------------------------------------------
import torch, numpy as np
from models.cnn import SimpleCNN
from datasets.loader import load_mnist
from client.client_node import ClientNode
from aggregator.server import Aggregator
from secure_agg.secagg import SecureAggregator
from ipfs.ipfs_client import IPFSClient
from chain.contract import FLContract

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using:", device)

## 1️⃣ Load Dataset

In [2]:
train_loaders, test_loader = load_mnist(num_clients=3, batch_size=32)
print("Loaded dataset for 3 clients.")

## 2️⃣ Initialize Clients

In [3]:
clients = []
for i in range(3):
    model = SimpleCNN().to(device)
    client = ClientNode(id=i, model=model, train_loader=train_loaders[i])
    clients.append(client)

print("Clients initialized.")

## 3️⃣ Local Training

In [4]:
client_updates = []
for c in clients:
    update = c.train_one_round(epochs=1)
    client_updates.append(update)

print("Local training done.")

## 4️⃣ Secure Aggregation

In [5]:
secagg = SecureAggregator()
secure_updates = secagg.aggregate(client_updates)
print("Secure aggregation complete.")

## 5️⃣ Upload to IPFS

In [6]:
ipfs = IPFSClient()
cid = ipfs.upload_json(secure_updates)
print("Uploaded to IPFS:", cid)

## 6️⃣ Commit Hash to Blockchain

In [7]:
contract = FLContract()
tx_hash = contract.commit_update(cid)
print("Blockchain TX Hash:", tx_hash)

## 7️⃣ Global Aggregation

In [8]:
aggregator = Aggregator(model=SimpleCNN().to(device))
global_model = aggregator.update_global_model(secure_updates)
print("Global model updated.")

# 🎉 Done!
This notebook shows the end-to-end decentralized federated learning workflow.